In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
file_path = Path(".csv")
df_crypto_model = pd.read_csv(file_path)

# Review the DataFrame
df_crypto_model.head()